In [1]:
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
from VLC_init import *
import scipy.fftpack
import math


In [2]:
"""
*: coordinate center of cari

|--------|                     
| car1   |                      
|-------*|
         |
       y |                   |---------|
         |                   |  car2   |
         |-------------------|*--------|
                    d

"""
def translate(self, value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin
    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)
    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

In [3]:
vlc_obj = VLC_init()
vlc_obj.update_lookuptable()
print(vlc_obj.H)
print(vlc_obj.eps_a)
print(vlc_obj.eps_b)
print(vlc_obj.eps_c)
print(vlc_obj.eps_d)

[[1.30566923e-10 9.19293971e-11]
 [2.41169141e-10 1.30566923e-10]]
[[0.24881092 0.24938314]
 [0.24831119 0.24881092]]
[[0.25118908 0.25061686]
 [0.25168881 0.25118908]]
[[0.24881092 0.24938314]
 [0.24831119 0.24881092]]
[[0.25118908 0.25061686]
 [0.25168881 0.25118908]]


In [4]:
f_m1 = 1000000  # 1MHz 
f_m2 = 2000000 # 2MHz 

dt = 5e-6; 
t = np.arange(0, 1e-2-dt, dt);
w1 = 2 * math.pi * f_m1
w2 = 2 * math.pi * f_m2
noise_standard_deviation = 1e-5

vlc_params = VLC_init()

delta_delay1 = (vlc_params.delays)[0][0] - (vlc_params.delays)[0][1]
delta_delay2 = vlc_params.delays[1][0] - vlc_params.delays[1][1]
s1_w1 = np.cos(w1*(t))
s1_w1 = s1_w1[:, np.newaxis].T
s2_w2 = np.cos(w2*(t))
s2_w2 = s2_w2[:, np.newaxis].T

#after going through ADC at receiver
r1_w1 = (vlc_obj.H[0][0]*np.cos(w1*(t - delta_delay1))+noise_standard_deviation*np.random.randn(1,len(t)))
r2_w1 = (vlc_obj.H[1][0]*np.cos(w1*(t))+noise_standard_deviation*np.random.randn(1,len(t)))

r1_w2 = (vlc_obj.H[0][1]*np.cos(w2*(t - delta_delay2))+noise_standard_deviation*np.random.randn(1,len(t)))
r2_w2 = (vlc_obj.H[1][1]*np.cos(w2*(t))+noise_standard_deviation*np.random.randn(1,len(t)))
t.shape

(1999,)

In [5]:
eps_a_s1, eps_b_s1, eps_c_s1, eps_d_s1, phi_h_s1 = np.array([0.,0.]), np.array([0.,0.]), np.array([0.,0.]), np.array([0.,0.]), np.array([0.,0.])
eps_a_s2, eps_b_s2, eps_c_s2, eps_d_s2, phi_h_s2 = np.array([0.,0.]), np.array([0.,0.]), np.array([0.,0.]), np.array([0.,0.]), np.array([0.,0.])

hbuf = 1000 
w0 = 500
theta_R_L = np.array([[0., 0.], [0., 0.]]).astype(longfloat)
u_q = 1e-12

for w in range(w0, (w0+hbuf-1)):
    eps_a_s1[0] += ((vlc_obj.eps_a[0][0]*r1_w1[0][w] - u_q)*s1_w1[0][w]) 
    eps_b_s1[0] += ((vlc_obj.eps_b[0][0]*r1_w1[0][w]- u_q)*s1_w1[0][w]) 
    eps_c_s1[0] += ((vlc_obj.eps_c[0][0]*r1_w1[0][w]- u_q)*s1_w1[0][w]) 
    eps_d_s1[0] += ((vlc_obj.eps_d[0][0]*r1_w1[0][w]- u_q)*s1_w1[0][w])
    eps_a_s1[1] += ((vlc_obj.eps_a[0][1]*r2_w1[0][w]- u_q)*s1_w1[0][w]) 
    eps_b_s1[1] += ((vlc_obj.eps_b[0][1]*r2_w1[0][w]- u_q)*s1_w1[0][w])
    eps_c_s1[1] += ((vlc_obj.eps_c[0][1]*r2_w1[0][w]- u_q)*s1_w1[0][w]) 
    eps_d_s1[1] += ((vlc_obj.eps_d[0][1]*r2_w1[0][w]- u_q)*s1_w1[0][w]) 
    
    eps_a_s2[0] += ((vlc_obj.eps_a[1][0]*r1_w2[0][w]- u_q)*s2_w2[0][w])
    eps_b_s2[0] += ((vlc_obj.eps_b[1][0]*r1_w2[0][w]- u_q)*s2_w2[0][w])
    eps_c_s2[0] += ((vlc_obj.eps_c[1][0]*r1_w2[0][w]- u_q)*s2_w2[0][w])
    eps_d_s2[0] += ((vlc_obj.eps_d[1][0]*r1_w2[0][w]- u_q)*s2_w2[0][w])
    eps_a_s2[1] += ((vlc_obj.eps_a[1][1]*r2_w2[0][w]- u_q)*s2_w2[0][w])
    eps_b_s2[1] += ((vlc_obj.eps_b[1][1]*r2_w2[0][w]- u_q)*s2_w2[0][w])
    eps_c_s2[1] += ((vlc_obj.eps_c[1][1]*r2_w2[0][w]- u_q)*s2_w2[0][w])
    eps_d_s2[1] += ((vlc_obj.eps_d[1][1]*r2_w2[0][w]- u_q)*s2_w2[0][w])


    
eps_a_s1[0] /= hbuf
eps_b_s1[0] /= hbuf
eps_c_s1[0] /= hbuf
eps_d_s1[0] /= hbuf
eps_a_s1[1] /= hbuf
eps_b_s1[1] /= hbuf
eps_c_s1[1] /= hbuf
eps_d_s1[1] /= hbuf

eps_a_s2[0] /= hbuf
eps_b_s2[0] /= hbuf
eps_c_s2[0] /= hbuf
eps_d_s2[0] /= hbuf
eps_a_s2[1] /= hbuf
eps_b_s2[1] /= hbuf
eps_c_s2[1] /= hbuf
eps_d_s2[1] /= hbuf


phi_h_s1[0] = ((eps_b_s1[0] + eps_d_s1[0]) - (eps_a_s1[0] + eps_c_s1[0])) / (eps_a_s1[0] + eps_b_s1[0] + eps_c_s1[0] + eps_d_s1[0])
phi_h_s1[1] = ((eps_b_s1[1] + eps_d_s1[1]) - (eps_a_s1[1] + eps_c_s1[1])) / (eps_a_s1[1] + eps_b_s1[1] + eps_c_s1[1] + eps_d_s1[1])
phi_h_s2[0] = ((eps_b_s2[0] + eps_d_s2[0]) - (eps_a_s2[0] + eps_c_s2[0])) / (eps_a_s2[0] + eps_b_s2[0] + eps_c_s2[0] + eps_d_s2[0])
phi_h_s2[1] = ((eps_b_s2[1] + eps_d_s2[1]) - (eps_a_s2[1] + eps_c_s2[1])) / (eps_a_s2[1] + eps_b_s2[1] + eps_c_s2[1] + eps_d_s2[1])



theta_R_L[0][0] = vlc_obj.translate(phi_h_s1[0], -1, 1, (-1*vlc_obj.e_angle), vlc_obj.e_angle)
theta_R_L[0][1] = vlc_obj.translate(phi_h_s1[1], -1, 1, (-1*vlc_obj.e_angle), vlc_obj.e_angle)
theta_R_L[1][0] = vlc_obj.translate(phi_h_s2[0], -1, 1, (-1*vlc_obj.e_angle), vlc_obj.e_angle)
theta_R_L[1][1] = vlc_obj.translate(phi_h_s2[1], -1, 1, (-1*vlc_obj.e_angle), vlc_obj.e_angle)


In [6]:
diff_1 = theta_R_L[0][1]-theta_R_L[0][0]
t_x_1 = vlc_obj.distancecar*(0.5 + (math.sin(theta_R_L[0][0])*math.cos(theta_R_L[0][1]))/(math.sin(diff_1)))
t_y_1 = vlc_obj.distancecar*((math.cos(theta_R_L[0][0])*math.cos(theta_R_L[0][1]))/(math.sin(diff_1)))
print("Transmitter-1 x pos is : ", t_x_1, ", y pos is : ", t_y_1)

diff_2 = theta_R_L[1][1]-theta_R_L[1][0]
t_x_2 = vlc_obj.distancecar*(0.5 + (math.sin(theta_R_L[1][0])*math.cos(theta_R_L[1][1]))/(math.sin(diff_2)))
t_y_2 = vlc_obj.distancecar*((math.cos(theta_R_L[1][0])*math.cos(theta_R_L[1][1]))/(math.sin(diff_2)))
print("Transmitter-2 x pos is : ", t_x_2, ", y pos is : ", t_y_2)




Transmitter-1 x pos is :  -1.4999783597749459 , y pos is :  -5.000031364095184
Transmitter-2 x pos is :  -2.50068417717565 , y pos is :  -5.001846627570651


In [7]:
#Error calc
t_x_1_act, t_y_1_act, t_x_2_act, t_y_2_act = vlc_obj.change_cords()

print("Error of Transmitter-1 position in x:", abs(t_x_1_act-t_x_1),", y:", abs(t_y_1_act-t_y_1))
print("Error of Transmitter-2 position in x, y:", abs(t_x_2_act-t_x_2),", y:", abs(t_y_2_act-t_y_2))




Error of Transmitter-1 position in x: 2.1640225054131434e-05 , y: 7.000031364095184
Error of Transmitter-2 position in x, y: 5.50068417717565 , y: 2.501846627570651
